<a href="https://dacon.io/competitions/official/136/overview/description/">대회링크</a>

# 대출 상점 총 매출 예측 경진대회

한국의 약 2,000개 상점의 신용카드 거래 내역이 제공됩니다. card_id를 기준으로 샘플링되었으며, amount의 단위는 KRW가 아닙니다. 테스트 파일에서 각 상점의 마지막 매출 발생일 다음 날부터 100일 후까지 매출의 총합을 예측해야 합니다.

 

예측한 금액만큼 100일간 대출을 진행합니다. 만약 예측한 금액보다 실제 매출이 적게 발생한다면 원금에 손실을 보게 됩니다. 실제 매출이 예측한 금액 이상으로 발생한다면, 연이율 13%의 이자가 발생합니다.



[Data]

train.csv - 카드 매출 내역, 2016-08-01부터 2018-07-31까지.


test.csv - train.csv랑 같은 형식. test.csv랑 train.csv의 store_id가 같아도 같은 상점인 것은 아님.

(월에 최소 5일 이상 데이터가 있는 상점만을 문제에 포함.)

- store_id : 각 파일에서의 상점 고유 번호 (해당기한 내 개업 혹은 폐업한 상점도 있음)

- date: 거래 일자
- time : 거래 시간
- card_id : 카드 번호의 hash 값
- amount : 매출 금액, 음수인 경우 취소 거래
- installments : 할부 개월 수. 일시불은 빈 문자열
- days_of_week : 요일, 월요일이 0, 일요일은 6
- holiday: 1이면 공휴일, 0이면 공휴일 아님


submission.csv – store_id와 total_sales 두 개의 열로 구성. store_id는 test.csv의 store_id와 대응.

# 접근방법

1. 가게마다 주 단위로 자른다
2. 주단위로 다음 주의 값을 예측한다

In [51]:
import pandas as pd
import missingno as msno
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from datetime import timedelta
import math

In [30]:
train = pd.read_csv('C:\kaggle_data/credit/train.csv')
test = pd.read_csv('C:\kaggle_data/credit/test.csv')
ss = pd.read_csv('C:\kaggle_data/credit/submission.csv')

In [31]:
train['amount'] = np.log(train['amount'])
train['amount'] = train['amount'].replace([np.inf, -np.inf], np.nan)

In [32]:
test['amount'] = np.log(test['amount'])
test['amount'] = test['amount'].replace([np.inf, -np.inf], np.nan)

# date는 datetime객체로 바꿔주자


In [33]:
def date_idx(df):
    df['date_idx'] = pd.DatetimeIndex(df['date'])
    df = df.set_index('date_idx')
    return df

In [34]:
train = date_idx(train)
test = date_idx(test)

In [35]:
def resam_1w(df):
    df = df.drop(['card_id','time','days_of_week','holyday','installments'],1)
    df['date_idx'] = pd.DatetimeIndex(df['date'])
    df = df.set_index('date_idx')
    store_lst = []
    for store_id in df['store_id'].unique():
    
        temp = df[df['store_id']==store_id]
        temp_0 = temp['amount'].resample('1W').sum()
        temp_0 = pd.DataFrame(temp_0)
        temp_0['target'] = temp_0['amount'].shift(-1).fillna(0).astype('int')
        temp_0 = temp_0.reset_index()
        
        
        temp_0['year'], temp_0['month'], temp_0['day'] = 0,0,0
        for i in range(len(temp_0)):
            temp_0.loc[i,'year'] = temp_0['date_idx'][i].year
            temp_0.loc[i,'month'] = temp_0['date_idx'][i].month
            temp_0.loc[i, 'day'] = temp_0['date_idx'][i].day
        temp_0['store_id'] = store_id
        temp_0['counts'] = len(temp_0)
        store_lst.append(temp_0)
        
    return pd.concat(store_lst)

In [36]:
%%time
train = resam_1w(train)
test = resam_1w(test)

Wall time: 1min 39s


In [37]:
train = train.drop('date_idx',1)

In [38]:
train['year'] = train['year'] - 2016
test['year'] = test['year'] - 2016

In [39]:
train['day'] = train['day']/31
test['day'] = test['day']/31

In [40]:
y = train.pop('target')

In [41]:
train = train[['counts', 'year', 'month', 'day','amount' ]]

In [42]:
import xgboost as xgb
# from sklearn.model_selection import KFold

xgb_reg = xgb.XGBRegressor(random_state=1123, tree_method='gpu_hist')
xgb_reg.fit(train, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=0,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan,
             monotone_constraints='(0,0,0,0,0)', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, objective='reg:squarederror',
             random_state=1123, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             subsample=1, tree_method='gpu_hist', validate_parameters=1,
             verbosity=None)

In [82]:
for s_id in test['store_id'].unique():
    print(s_id)
    temp = test[test['store_id']==s_id]
    for i in range(14):
        last_row = temp.tail(1)
        counts = last_row['counts'].values[0]
        store_id = last_row['store_id'].values[0]
        next_date = (last_row['date_idx'] + timedelta(days=7)).iloc[-1]
        target = xgb_reg.predict(last_row[['counts', 'year', 'month', 'day', 'amount']])
        temp = temp.append({'date_idx':next_date, 
                     'amount':target[0], 
                     'year':next_date.year-2016,
                     'month':next_date.month,
                     'day':next_date.day/31,
                     'store_id': store_id,
                     'counts':counts}, ignore_index=True)
    x = temp.iloc[-14:]['amount']
    x = np.exp(x)
    x = sum(x)
    if x == np.inf:
        x = 0
    else:
        x = int(float(str(x)[:4]))

    
    ss.iloc[s_id][1] = x

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [83]:
ss.to_csv('baseline+log.csv',index=False)

In [49]:
x = 3.557145051407635

In [61]:
math.floor(x)

3667182614420377064420660294981292916736

In [65]:
np.trunc(x)

3.667182614420377e+39

In [71]:
x

3.667182614420377e+39

In [70]:
int(str(x)[:1])

3

In [76]:
s_id = 23
temp = test[test['store_id']==s_id]
for i in range(14):
    last_row = temp.tail(1)
    
    counts = last_row['counts'].values[0]
    store_id = last_row['store_id'].values[0]
    next_date = (last_row['date_idx'] + timedelta(days=7)).iloc[-1]
    target = xgb_reg.predict(last_row[['counts', 'year', 'month', 'day', 'amount']])
    
    temp = temp.append({'date_idx':next_date, 
                 'amount':target[0], 
                 'year':next_date.year-2016,
                 'month':next_date.month,
                 'day':next_date.day/31,
                 'store_id': store_id,
                 'counts':counts}, ignore_index=True)
    
    x = temp.iloc[-14:]['amount']
    print('zz5')
    x = np.exp(x)
    print('zz6')
    x = sum(x)
    print('zz7')
    if x == np.inf:
        x = 0
    else:
        
    ss.iloc[s_id][1] = x

zz5
zz6
zz7


ValueError: invalid literal for int() with base 10: 'inf'

In [80]:
x == np.inf

True